<a href="https://colab.research.google.com/github/Amplil/rubikcube/blob/master/is_a_learning_NaiveBayes%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E6%A7%8B%E7%AF%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#初期化

In [2]:
import pandas as pd
import numpy as np

In [3]:
class sentence(list): # オブジェクトでリスト表記が使えるようにする
    common_dict={}
    last_value=0
    def __init__(self,original_sentence):
        self.extend(original_sentence.split()) # リストの連結
    def __eq__(self,other_sentence):
        common=[]
        for i,l1 in enumerate(self):
            for j,l2 in enumerate(other_sentence):
                if(l1==l2):
                    common.append([i,j])
                    sentence.add_common(str([i,j])) # 共通部分を個別に辞書登録
        return str(common)
    @classmethod
    def add_common(cls,common):
        if cls.common_dict.get(common)==None:
            cls.last_value+=1
            cls.common_dict[common]=cls.last_value
        return cls.common_dict
    
class SentenceUnit(list): # オブジェクトでリスト表記が使えるようにする
    word_dict={}
    last_value=0
    def __init__(self,sentence_list):
        [self.append(sentence(s)) for s in sentence_list] # リスト内包表記 
        for s in self:
            for word in s:
                SentenceUnit.add_word(word)
    @classmethod
    def add_word(cls,word):
        if cls.word_dict.get(word)==None:
            cls.last_value+=1
            cls.word_dict[word]=cls.last_value
        return cls.word_dict

# ナイーブベイズ
def pword_cat(word,cat):
    catBoW=data[data['a[n][0]'].isin([cat])] # category BoW
    return (catBoW[word].sum()/catBoW.sum(axis=1).sum())
def pcat(cat):
    catBoW=data[data['a[n][0]'].isin([cat])] # category BoW
    return len(catBoW.index)/len(data.index)

## 前提premise(知識)は最初から読み込む

In [4]:
p=SentenceUnit(['John is-a human','bird is-a animal','human is-a animal'])
p

[['John', 'is-a', 'human'],
 ['bird', 'is-a', 'animal'],
 ['human', 'is-a', 'animal']]

## 入力: 質問（question）

In [5]:
q=SentenceUnit(['is John human ?','is bird animal ?','is not John human ?','is not bird animal ?'])
q

[['is', 'John', 'human', '?'],
 ['is', 'bird', 'animal', '?'],
 ['is', 'not', 'John', 'human', '?'],
 ['is', 'not', 'bird', 'animal', '?']]

## 出力: 答え（answer）

In [6]:
a=SentenceUnit(['Yes','Yes','No','No'])
a

[['Yes'], ['Yes'], ['No'], ['No']]

## 理由(reason)

In [7]:
reason_m=[0,1,0,1]

# 質問と答えのモデルで作ったBoW(エンコード)

In [11]:
qn_pm=[q[n]==p[reason_m[n]] for n in range(len(q))]
qn_pm

['[[1, 0], [2, 2]]',
 '[[1, 0], [2, 2]]',
 '[[2, 0], [3, 2]]',
 '[[2, 0], [3, 2]]']

In [12]:
sentence.common_dict

{'[1, 0]': 1, '[2, 0]': 3, '[2, 2]': 2, '[3, 2]': 4}

In [31]:
data_dict={}
for word in SentenceUnit.word_dict:
    data_dict[word]=[s.count(word) for s in q]
for common in sentence.common_dict:
    data_dict[common]=[(1 if (common in qn_pm[n]) else 0) for n in range(len(q))]
data_dict["a[n][0]"]=[a[n][0] for n in range(4)]
data_dict["a[n][1]"]=[None for n in range(4)]
data = pd.DataFrame(data_dict)
data

,John,is-a,human,bird,animal,is,?,not,Yes,No,"[1, 0]","[2, 2]","[2, 0]","[3, 2]",a[n][0],a[n][1]
0,1,0,1,0,0,1,1,0,0,0,1,1,0,0,Yes,None
1,0,0,0,1,1,1,1,0,0,0,1,1,0,0,Yes,None
2,1,0,1,0,0,1,1,1,0,0,0,0,1,1,No,None
3,0,0,0,1,1,1,1,1,0,0,0,0,1,1,No,None


# ナイーブベイズによる答えの導出

## ナイーブベイズの計算過程

In [14]:
an0=data['a[n][0]'].value_counts()
for cat in an0.index:
    catBoW=data[data['a[n][0]'].isin([cat])] # category BoW
    for word in data.columns[:-2]:
        print("p(",word,"|",cat,")p(",cat,")=(",catBoW[word].sum(),"/",catBoW.sum(axis=1).sum(),")*",len(catBoW.index),"/",len(data.index),"=",(catBoW[word].sum()/catBoW.sum(axis=1).sum())*len(catBoW.index)/len(data.index))

p( John | No )p( No )=( 1 / 14 )* 2 / 4 = 0.03571428571428571
p( is-a | No )p( No )=( 0 / 14 )* 2 / 4 = 0.0
p( human | No )p( No )=( 1 / 14 )* 2 / 4 = 0.03571428571428571
p( bird | No )p( No )=( 1 / 14 )* 2 / 4 = 0.03571428571428571
p( animal | No )p( No )=( 1 / 14 )* 2 / 4 = 0.03571428571428571
p( is | No )p( No )=( 2 / 14 )* 2 / 4 = 0.07142857142857142
p( ? | No )p( No )=( 2 / 14 )* 2 / 4 = 0.07142857142857142
p( not | No )p( No )=( 2 / 14 )* 2 / 4 = 0.07142857142857142
p( Yes | No )p( No )=( 0 / 14 )* 2 / 4 = 0.0
p( No | No )p( No )=( 0 / 14 )* 2 / 4 = 0.0
p( [1, 0] | No )p( No )=( 0 / 14 )* 2 / 4 = 0.0
p( [2, 2] | No )p( No )=( 0 / 14 )* 2 / 4 = 0.0
p( [2, 0] | No )p( No )=( 2 / 14 )* 2 / 4 = 0.07142857142857142
p( [3, 2] | No )p( No )=( 2 / 14 )* 2 / 4 = 0.07142857142857142
p( John | Yes )p( Yes )=( 1 / 12 )* 2 / 4 = 0.041666666666666664
p( is-a | Yes )p( Yes )=( 0 / 12 )* 2 / 4 = 0.0
p( human | Yes )p( Yes )=( 1 / 12 )* 2 / 4 = 0.041666666666666664
p( bird | Yes )p( Yes )=( 1 / 1

In [32]:
# 'is not John human ?'の場合
list(data[2:3].loc[:,(data[2:3]!=0).any(axis=0)].columns)[:-2]

['John', 'human', 'is', '?', 'not', '[2, 0]', '[3, 2]']

In [17]:
# 'is not John human ?'の場合
text=list(data[2:3].loc[:,(data[2:3]!=0).any(axis=0)].columns)[:-2]
for word in text:
    print('p(',word,'| category=Yes)=',pword_cat(word,'Yes'))
print('p(text|category=Yes.)p(category=Yes.)=',np.prod([pword_cat(wordi,'Yes') for wordi in text])*pcat('Yes'),'\n')
for word in text:
    print('p(',word,'| category=No)=',pword_cat(word,'No'))
print('p(text|category=No.)p(category=No.)=',np.prod([pword_cat(wordi,'No') for wordi in text])*pcat('No'))

p( John | category=Yes)= 0.08333333333333333
p( human | category=Yes)= 0.08333333333333333
p( is | category=Yes)= 0.16666666666666666
p( ? | category=Yes)= 0.16666666666666666
p( not | category=Yes)= 0.0
p( [2, 0] | category=Yes)= 0.0
p( [3, 2] | category=Yes)= 0.0
p(text|category=Yes.)p(category=Yes.)= 0.0 

p( John | category=No)= 0.07142857142857142
p( human | category=No)= 0.07142857142857142
p( is | category=No)= 0.14285714285714285
p( ? | category=No)= 0.14285714285714285
p( not | category=No)= 0.14285714285714285
p( [2, 0] | category=No)= 0.14285714285714285
p( [3, 2] | category=No)= 0.14285714285714285
p(text|category=No.)p(category=No.)= 1.517832098627515e-07


In [34]:
pword_cat(word,'Yes')

0.0

In [36]:
# 'is not John human ?'の場合
text=list(data[2:3].loc[:,(data[2:3]!=0).any(axis=0)].columns)[:-2]
for word in text:
    print('p(',word,'| category=Yes)=',pword_cat(word,'Yes'))
print('p(text|category=Yes.)p(category=Yes.)=',np.prod([pword_cat(wordi,'Yes') for wordi in text])*pcat('Yes'),'\n')
for word in text:
    print('p(',word,'| category=No)=',pword_cat(word,'No'))
print('p(text|category=No.)p(category=No.)=',np.prod([pword_cat(wordi,'No') for wordi in text])*pcat('No'))

p( John | category=Yes)= 0.08333333333333333
p( human | category=Yes)= 0.08333333333333333
p( is | category=Yes)= 0.16666666666666666
p( ? | category=Yes)= 0.16666666666666666
p( not | category=Yes)= 0.0
p( [2, 0] | category=Yes)= 0.0
p( [3, 2] | category=Yes)= 0.0
p(text|category=Yes.)p(category=Yes.)= 0.0 

p( John | category=No)= 0.07142857142857142
p( human | category=No)= 0.07142857142857142
p( is | category=No)= 0.14285714285714285
p( ? | category=No)= 0.14285714285714285
p( not | category=No)= 0.14285714285714285
p( [2, 0] | category=No)= 0.14285714285714285
p( [3, 2] | category=No)= 0.14285714285714285
p(text|category=No.)p(category=No.)= 1.517832098627515e-07


In [13]:
list(data[1:2].loc[:,(data[1:2]!=0).any(axis=0)].columns)[:-2] # 'is bird animal ?'の場合

['bird', 'animal', 'is', '?', '[1, 0]', '[2, 2]']

In [15]:
# 'is bird animal ?'の場合
text=list(data[1:2].loc[:,(data[1:2]!=0).any(axis=0)].columns)[:-2]
for word in text:
    print('p(',word,'| category=Yes)=',pword_cat(word,'Yes'))
print('p(text|category=Yes.)p(category=Yes.)=',np.prod([pword_cat(wordi,'Yes') for wordi in text])*pcat('Yes'),'\n')
for word in text:
    print('p(',word,'| category=No)=',pword_cat(word,'No'))
print('p(text|category=No.)p(category=No.)=',np.prod([pword_cat(wordi,'No') for wordi in text])*pcat('No'))

p( bird | category=Yes)= 0.08333333333333333
p( animal | category=Yes)= 0.08333333333333333
p( is | category=Yes)= 0.16666666666666666
p( ? | category=Yes)= 0.16666666666666666
p( [1, 0] | category=Yes)= 0.16666666666666666
p( [2, 2] | category=Yes)= 0.16666666666666666
p(text|category=Yes.)p(category=Yes.)= 2.679183813443072e-06 

p( bird | category=No)= 0.07142857142857142
p( animal | category=No)= 0.07142857142857142
p( is | category=No)= 0.14285714285714285
p( ? | category=No)= 0.14285714285714285
p( [1, 0] | category=No)= 0.0
p( [2, 2] | category=No)= 0.0
p(text|category=No.)p(category=No.)= 0.0


## ナイーブベイズによる出力

In [39]:
text=list(data[2:3].loc[:,(data[2:3]!=0).any(axis=0)].columns)[:-2] # 'is not John human ?'の場合を検証
NaiveBayes_yes=np.prod([pword_cat(wordi,'Yes') for wordi in text])*pcat('Yes')
NaiveBayes_no=np.prod([pword_cat(wordi,'No') for wordi in text])*pcat('No')
if NaiveBayes_yes>NaiveBayes_no:
    NaiveBayes_ans='Yes'
else:
    NaiveBayes_ans='No'

NaiveBayes_ans

'No'

In [16]:
text=list(data[1:2].loc[:,(data[1:2]!=0).any(axis=0)].columns)[:-2] # 'is bird animal ?'の場合を検証
NaiveBayes_yes=np.prod([pword_cat(wordi,'Yes') for wordi in text])*pcat('Yes')
NaiveBayes_no=np.prod([pword_cat(wordi,'No') for wordi in text])*pcat('No')
if NaiveBayes_yes>NaiveBayes_no:
    NaiveBayes_ans='Yes'
else:
    NaiveBayes_ans='No'

NaiveBayes_ans

'Yes'

# 学習から検証までをすべてまとめる

In [9]:
# 共通部分
qn_pm=[q[n]==p[reason_m[n]] for n in range(len(q))] 

# BoW(エンコード)
data_dict={}
for word in SentenceUnit.word_dict:
    data_dict[word]=[s.count(word) for s in q]
for common in sentence.common_dict:
    data_dict[common]=[(1 if (common in qn_pm[n]) else 0) for n in range(len(q))]
data_dict["a[n][0]"]=[a[n][0] for n in range(4)]
data_dict["a[n][1]"]=[None for n in range(4)]
data = pd.DataFrame(data_dict) 

text=list(data[2:3].loc[:,(data[2:3]!=0).any(axis=0)].columns)[:-2] # 'is not John human ?'の場合を検証

# ナイーブベイズ
NaiveBayes_yes=np.prod([pword_cat(wordi,'Yes') for wordi in text])*pcat('Yes')
NaiveBayes_no=np.prod([pword_cat(wordi,'No') for wordi in text])*pcat('No')
if NaiveBayes_yes>NaiveBayes_no:
    NaiveBayes_ans='Yes'
else:
    NaiveBayes_ans='No'

NaiveBayes_ans

'No'